# Code

In [1]:
code = """
(declare-rel inv (Int Int ))
(declare-var x0 Int)
(declare-var x1 Int)
(declare-var y0 Int)
(declare-var y1 Int)
(declare-var z1 Int)

(declare-rel fail ())

(rule (=> (and (= x0 0) (= y0 5000))
    (inv x0 y0)))

(rule (=> (and
        (inv x0 y0)
        (= x1 (+ x0 1))
        (= y1 (ite (>= x0 5000) (+ y0 1) y0)))
    (inv x1 y1)))

(rule (=> (and (inv x0 y0) (= x0 10000)
    (not (= y0 x0))) fail))

(query fail)
"""

# Helpers

In [2]:
import z3
import os
import sys
module_path = os.path.abspath(os.path.join('../../../chc-tools'))
if module_path not in sys.path:
    sys.path.append(module_path)
import chctools

# build some formulas for testing
x, y, z = z3.Ints('x y z')
phi = z3.ForAll(x, z3.Exists([y, z], z3.Implies(x > 0, z3.And(y > 1, z > 4))))
phi

ForAll(x, Exists([y, z], Implies(x > 0, And(y > 1, z > 4))))

In [3]:
z3.Const(phi.var_name(0), phi.var_sort(0))


x

In [4]:
phix = z3.substitute_vars(phi.body(), x)
z3.substitute_vars(phix.body(), *reversed([y, z]))

Implies(x > 0, And(y > 1, z > 4))

In [5]:
def expand_quant(fml):
    """ Expands quantifier into Quantifier, Variables, and Body
    
        The result is a triple (Q, vars, body), such that Q(vars, body) is equivalent to fml
        All variables in body are ground (i.e., regular constants)
    """
    if z3.is_quantifier(fml):
        gnd_vars = [z3.Const(fml.var_name(i), fml.var_sort(i)) for i in range(fml.num_vars())]
        gnd_body = z3.substitute_vars(fml.body(), *reversed(gnd_vars))
        quant = z3.Lambda
        if fml.is_exists():
            quant = z3.Exists
        elif fml.is_forall():
            quant = z3.ForAll
        return quant, gnd_vars, gnd_body
    else:
        return (lambda x, y : y), [], fml


# some test
q, v, b = expand_quant(phi.body())
q, v, b

(<function z3.z3.Exists(vs, body, weight=1, qid='', skid='', patterns=[], no_patterns=[])>,
 [y, z],
 Implies(Var(2) > 0, And(y > 1, z > 4)))

In [6]:
q(v, b)

Exists([y, z], Implies(Var(2) > 0, And(y > 1, z > 4)))

In [7]:
def for_each_expr(fml, fn, *args, **kwargs):
    """ Execute given function fn on every sub-expression 
    
        The return value of fn is used to decided whether children should be explored

        Additional arguments are passed to fn and can be used to maintain state
    """

    do_kids = fn(fml, *args, **kwargs)

    if not do_kids: return
    for k in fml.children():
        for_each_expr(k, fn, *args, **kwargs)


# example of a function that records every expression visited
def myfn(fml, visited):
    visited.add(fml.get_id())
    return True

visited = set()
for_each_expr(phi.body().body(), myfn, visited)
print(visited)

{8, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20}


In [8]:
# an example of a function that records every expresison visited
# and never visits an expression more than once (i.e., caches what it visited)
def myfn2(fml, visited):
    if fml.get_id() in visited:
        return False
    
    # visiting fml the first time
    visited.add(fml.get_id())
    return True

visited = set()
for_each_expr(phi.body().body(), myfn2, visited)
print(visited)

{8, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20}


# TOOL

In [9]:
import z3
fp = z3.Fixedpoint()
queries = fp.parse_string(code)
fp.set('spacer.max_level', 40)
fp.query(queries[0])

unknown

In [10]:
print(fp.to_string(queries))

(declare-rel fail ())
(declare-rel inv (Int Int))
(declare-var A Int)
(declare-var B Int)
(declare-var C Int)
(declare-var D Int)
(rule (=> (and (= B 0) (= A 5000)) (inv B A)))
(rule (let ((a!1 (and (inv D C) (= B (+ D 1)) (= A (ite (>= D 5000) (+ C 1) C)))))
  (=> a!1 (inv B A))))
(rule (=> (and (inv A B) (= A 10000) (not (= B A))) fail))
(query fail)



In [11]:
rules = fp.get_rules()
rules[0]

ForAll([A, B], Implies(And(A == 0, B == 5000), inv(A, B)))

In [12]:
rules

[ForAll([A, B], Implies(And(A == 0, B == 5000), inv(A, B))),
 ForAll([A, B, C, D],
        Implies(And(inv(A, B),
                    C == A + 1,
                    D == If(A >= 5000, B + 1, B)),
                inv(C, D))),
 ForAll([A, B],
        Implies(And(inv(B, A), B == 10000, Not(A == B)),
                fail))]

In [13]:
r = rules[0]
r, r.body().sexpr(), [x.sexpr() for x in r.body().children()]

(ForAll([A, B], Implies(And(A == 0, B == 5000), inv(A, B))),
 '(=> (and (= (:var 1) 0) (= (:var 0) 5000)) (inv (:var 1) (:var 0)))',
 ['(and (= (:var 1) 0) (= (:var 0) 5000))', '(inv (:var 1) (:var 0))'])

In [14]:
num = r.body().arg(0).arg(1).arg(1)
z3.is_int(num), num.as_long(), num.as_string()

(True, 5000, '5000')

In [15]:
for rule in rules:
    rl = rule.body()
    # rl2 = z3.substitute(rl, (var(1), )
    print(f"rule={rule},\n rl={rl} \n\n")

rule=ForAll([A, B], Implies(And(A == 0, B == 5000), inv(A, B))),
 rl=Implies(And(Var(1) == 0, Var(0) == 5000),
        inv(Var(1), Var(0))) 


rule=ForAll([A, B, C, D],
       Implies(And(inv(A, B),
                   C == A + 1,
                   D == If(A >= 5000, B + 1, B)),
               inv(C, D))),
 rl=Implies(And(inv(Var(3), Var(2)),
            Var(1) == Var(3) + 1,
            Var(0) == If(Var(3) >= 5000, Var(2) + 1, Var(2))),
        inv(Var(1), Var(0))) 


rule=ForAll([A, B],
       Implies(And(inv(B, A), B == 10000, Not(A == B)),
               fail)),
 rl=Implies(And(inv(Var(0), Var(1)),
            Var(0) == 10000,
            Not(Var(1) == Var(0))),
        fail) 




In [16]:
N = z3.Int("N")
new_init = z3.ForAll(N, z3.Implies(N == num, z3.substitute(r, (num, N))))
new_init

ForAll(N,
       Implies(5000 == N,
               ForAll([A, B],
                      Implies(And(A == 0, B == N),
                              inv(A, B)))))

In [17]:
q, vars, gnd_body = expand_quant(r)
gnd_body = z3.Implies(N == num, z3.substitute(gnd_body, (num, N)))
gnd_body
print(vars)
print(vars + [N])
q(vars + [N], gnd_body)


[A, B]
[A, B, N]


ForAll([A, B, N],
       Implies(5000 == N,
               Implies(And(A == 0, B == N), inv(A, B))))

In [18]:
new_init.body()

Implies(5000 == Var(0),
        ForAll([A, B],
               Implies(And(A == 0, B == Var(2)), inv(A, B))))

In [19]:
def is_magic_num(v):
    return z3.is_int_value(v) and v.as_long() != 0

def has_comparison_operator(expr):
    comparison_ops = [
        z3.is_lt,
        z3.is_le,
        z3.is_gt,
        z3.is_ge,
        z3.is_eq,
        z3.is_distinct,
    ]

    if any(op(expr) for op in comparison_ops):
        return True

    return False

In [20]:
def find_magic_in_rule(rule):
    q, v, b = expand_quant(rule)
    return find_magic_in_gnd_rule(b)

def find_magic_in_gnd_rule(rule):
    myset = set()

    def find_magic(x, found):
        if has_comparison_operator(x): 
            f_arg = x.arg(0)
            s_arg = x.arg(1)

            if is_magic_num(f_arg):
                found.add(f_arg)
            if is_magic_num(s_arg):
                found.add(s_arg)

            return False
        else:
            return True
        
    for_each_expr(rule, find_magic, found=myset)
    return myset

In [21]:
[s1, s2, s3] = map(find_magic_in_rule, rules)
list(s1.union(s2, s3))

[10000, 5000]

In [22]:
# Define the values to substitute
values = [10000, 5000]

variables_to_substitute = [z3.Int(var) for var in [f"K{val}" for val in values]]
#variables_to_substitute = [z3.Var(val, z3.IntSort()) for val in values]

# Z3 constants for values
const_values = [z3.IntVal(val) for val in values]

# Create a dictionary for substitutions
substitutions = [(value, var) for var, value in zip(variables_to_substitute, const_values)]
print(f"substitutions={substitutions}")

fp_test = z3.Fixedpoint()
fp_test.declare_var(variables_to_substitute)
# Substitute variables in parsed rules and queries
s_rules = [z3.substitute(rule, substitutions) for rule in fp.get_rules()]
# print(s_rules)



#print(f"s_rules: {s_rules}")
print(f"s_rules[0]: {s_rules[0]}")
print(f"s_rules[1]: {s_rules[1]}")
print(f"s_rules[2]: {s_rules[2]}")
print("-----")

fp_s = z3.Fixedpoint()

#l=z3.Int(fp_s.var_name(0))

new_vars = [z3.IntVector("var", rule.num_vars())+variables_to_substitute for rule in s_rules]
new_bodies = [rule.body() for rule in s_rules]
new_bodies2 = [z3.substitute_vars(body, *variables) for body, variables in zip(new_bodies, new_vars)]
# print(f"new_vars={new_vars}")
# print(f"new_bodies2={new_bodies2}")
new_rules = [z3.ForAll(variables, body) for body, variables in zip(new_bodies2, new_vars)]

print(f"new_vars={new_vars}")
print(f"new_rules={new_rules}")

# fp_test.add_rule(s_rules[0])
# print(f"fp_test: {fp_test}")

# for s_rule in s_rules:
#     fp_s.add_rule(s_rule)
# print(fp_s)
# 
for s_rule in new_rules:
    fp_s.add_rule(s_rule)
print(fp_s)

#queries[0]
# print(fp_s.to_string(queries))
# import chctools.horndb
# db = load_horn_db_from_file("test", simplify_queries=False)
# db.load_from_fp(fp_s, queries)

# with open('res.smt2', 'w') as f:
#     print(fp_s.to_string(queries), file=f)  # Python 3.x

substitutions=[(10000, K10000), (5000, K5000)]
s_rules[0]: ForAll([A, B], Implies(And(A == 0, B == K5000), inv(A, B)))
s_rules[1]: ForAll([A, B, C, D],
       Implies(And(inv(A, B),
                   C == A + 1,
                   D == If(A >= K5000, B + 1, B)),
               inv(C, D)))
s_rules[2]: ForAll([A, B],
       Implies(And(inv(B, A), B == K10000, Not(A == B)),
               fail))
-----
new_vars=[[var__0, var__1, K10000, K5000], [var__0, var__1, var__2, var__3, K10000, K5000], [var__0, var__1, K10000, K5000]]
new_rules=[ForAll([var__0, var__1, K10000, K5000],
       Implies(And(var__1 == 0, var__0 == K5000),
               inv(var__1, var__0))), ForAll([var__0, var__1, var__2, var__3, K10000, K5000],
       Implies(And(inv(var__3, var__2),
                   var__1 == var__3 + 1,
                   var__0 ==
                   If(var__3 >= K5000, var__2 + 1, var__2)),
               inv(var__1, var__0))), ForAll([var__0, var__1, K10000, K5000],
       Implies(And(inv(var__

In [23]:
Z = z3.IntSort()
B = z3.BoolSort()
inv2_sorts = [Z, Z]
vrev = list(reversed(variables_to_substitute))
for v in vrev:
    inv2_sorts.append(v.sort())
inv2_sorts.append(B)
inv2_fdecl = z3.Function("Inv2", *inv2_sorts)
print(inv2_fdecl.sexpr())
fp_s.register_relation(inv2_fdecl)

inv2_args = [x, y] + vrev
inv2_term = inv2_fdecl(*inv2_args)
print(inv2_term.sexpr())

inv_fdecl = z3.Function('Inv', Z, Z, B)
inv_term= inv_fdecl(x, y)
print(inv_term.sexpr())

#z3.subsitute(fml, (inv_term, inv2_term))
inv2_term.decl().name()

(declare-fun Inv2 (Int Int Int Int) Bool)
(Inv2 x y K5000 K10000)
(Inv x y)


'Inv2'

In [24]:
def find_invs(gnd_rule_body):
    found = set()
    def is_inv_term(e, found):
        #print("Looking at ", e.sexpr())
        if e.decl().name() == 'inv':
            found.add(e)
            return False
        return True
    
    for_each_expr(gnd_rule_body, is_inv_term, found=found)
    return found

In [25]:
q, vars, body = expand_quant(s_rules[1])
inv_terms = find_invs(body)
inv_terms

{inv(A, B), inv(C, D)}

In [26]:
def mk_inv2_term(inv_term, new_vars):
    Z = z3.IntSort()
    B = z3.BoolSort()
    inv2_sorts = [inv_term.decl().domain(i) for i in range(inv_term.decl().arity())]
    vrev = list(reversed(new_vars))
    for v in vrev:
        inv2_sorts.append(v.sort())
    inv2_sorts.append(B)
    inv2_fdecl = z3.Function("inv2", *inv2_sorts)
    print(inv2_fdecl.sexpr())
    
    inv2_args = inv_term.children() + vrev
    inv2_term = inv2_fdecl(*inv2_args)
    print(inv2_term.sexpr())
    return inv2_term

subs = list()
for inv_term in inv_terms:
    # TODO: create inv2_term for inv_term
    inv2_term = mk_inv2_term(inv_term, variables_to_substitute)
    subs.append((inv_term, inv2_term))

new_body = z3.substitute(body, subs)
print(body.sexpr())
print(new_body.sexpr())
# for every ground body, apply z3.substitute with the above subs
new_rule = new_body

print("\n")
print(f"new_rule={new_rule}")

fp_new = z3.Fixedpoint()
fp_new.register_relation(z3.Function('inv2', Z, Z, Z, Z, B))
fp_new.declare_var(*vars)
fp_new.declare_var(*variables_to_substitute)
fp_new.add_rule(new_rule)
print(fp_new.sexpr())

(declare-fun inv2 (Int Int Int Int) Bool)
(inv2 C D K5000 K10000)
(declare-fun inv2 (Int Int Int Int) Bool)
(inv2 A B K5000 K10000)
(let ((a!1 (and (inv A B) (= C (+ A 1)) (= D (ite (>= A K5000) (+ B 1) B)))))
  (=> a!1 (inv C D)))
(let ((a!1 (and (inv2 A B K5000 K10000)
                (= C (+ A 1))
                (= D (ite (>= A K5000) (+ B 1) B)))))
  (=> a!1 (inv2 C D K5000 K10000)))


new_rule=Implies(And(inv2(A, B, K5000, K10000),
            C == A + 1,
            D == If(A >= K5000, B + 1, B)),
        inv2(C, D, K5000, K10000))
(declare-rel inv2 (Int Int Int Int))
(declare-var A Int)
(declare-var B Int)
(declare-var C Int)
(declare-var D Int)
(declare-var E Int)
(declare-var F Int)
(rule (let ((a!1 (and (inv2 F E A B) (= D (+ F 1)) (= C (ite (>= F A) (+ E 1) E)))))
  (=> a!1 (inv2 D C A B))))



In [27]:
a = z3.IntVal(100)
b = z3.Int('b')
c = z3.Int('b')
print(z3.solve(b == c))
print(f"b={b}, c={c}")
e = a + 5
s = [(a, b)]
z3.substitute(e, s)

[]
None
b=b, c=b


b + 5

In [28]:
q, v, b = expand_quant(new_init)
q, v, b

(<function z3.z3.ForAll(vs, body, weight=1, qid='', skid='', patterns=[], no_patterns=[])>,
 [N],
 Implies(5000 == N,
         ForAll([A, B],
                Implies(And(A == 0, B == N), inv(A, B)))))

In [29]:
q(v, b)

ForAll(N,
       Implies(5000 == N,
               ForAll([A, B],
                      Implies(And(A == 0, B == N),
                              inv(A, B)))))

In [30]:
v0 = new_init.body().arg(0).arg(1)
z3.is_var(v0)

True

In [31]:
n0 = b.arg(0).arg(1)
z3.is_var(n0)

False

In [32]:
print(fp.sexpr())

(declare-rel fail ())
(declare-rel inv (Int Int))
(declare-var A Int)
(declare-var B Int)
(declare-var C Int)
(declare-var D Int)
(rule (=> (and (= B 0) (= A 5000)) (inv B A)))
(rule (let ((a!1 (and (inv D C) (= B (+ D 1)) (= A (ite (>= D 5000) (+ C 1) C)))))
  (=> a!1 (inv B A))))
(rule (=> (and (inv A B) (= A 10000) (not (= B A))) fail))



In [33]:
fp2 = z3.Fixedpoint()
fp2.add_rule(new_init)
qrs = fp2.get_rules()

print(fp2)

(declare-fun inv (Int Int) Bool)
(rule (forall ((N Int))
  (let ((a!1 (forall ((A Int) (B Int))
               (! (=> (and (= A 0) (= B N)) (inv A B)) :weight 0))))
    (=> (= 5000 N) a!1))))

